# 作成するアプリケーション

## LGTM画像を自動生成するコマンドラインツール

In [1]:
%pip install git+https://github.com/rhoboro/lgtm#egg=lgtm

  Cloning https://github.com/rhoboro/lgtm to /private/var/folders/jp/08fw_dmn09q2rcvk5tjzn2fm0000gn/T/pip-install-3oa5tzrb/lgtm
  Running command git clone -q https://github.com/rhoboro/lgtm /private/var/folders/jp/08fw_dmn09q2rcvk5tjzn2fm0000gn/T/pip-install-3oa5tzrb/lgtm
     |████████████████████████████████| 81kB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 1.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 16.9MB/s eta 0:00:01
     |████████████████████████████████| 61kB 19.3MB/s eta 0:00:01
     |████████████████████████████████| 163kB 5.1MB/s eta 0:00:01
     |████████████████████████████████| 133kB 4.6MB/s eta 0:00:01
     |████████████████████████████████| 143kB 6.6MB/s eta 0:00:01
  Running setup.py install for lgtm ... done
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# ヘルプの表示
!lgtm --help

Usage: lgtm [OPTIONS] KEYWORD

  LGTM画像生成ツール

Options:
  -m, --message TEXT  画像に乗せる文字列  [default: LGTM]
  --help              Show this message and exit.


In [3]:
# 「book」で画像検索を行いoutput.pngを生成
# キーワードの代わりに画像パスや画像URLも指定できる
!lgtm book

## 利用する主なパッケージ

### requests ── HTTPクライアントライブラリ

In [4]:
# requestsのインストール
%pip install requests==2.22.0

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
from urllib import request, parse, error
import json
query = parse.urlencode({'q': 'python'})

# httpbinはリクエストの内容を返してくれる
url = f'https://httpbin.org/get?{query}'
try:
    with request.urlopen(url) as f:
        res = f.read().decode('utf-8')
except error.HTTPError as e:
    print(e)

json.loads(res)

{'args': {'q': 'python'},
 'headers': {'Accept-Encoding': 'identity',
  'Host': 'httpbin.org',
  'User-Agent': 'Python-urllib/3.8'},
 'origin': '153.182.176.137, 153.182.176.137',
 'url': 'https://httpbin.org/get?q=python'}

In [6]:
import requests
res = requests.get('https://httpbin.org/get',
                   params={'q': 'python'})
res.json()

{'args': {'q': 'python'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.22.0'},
 'origin': '153.182.176.137, 153.182.176.137',
 'url': 'https://httpbin.org/get?q=python'}

In [7]:
res = requests.post('https://httpbin.org/post',
                    data={'q': 'python'})

In [8]:
res.json()['form']

{'q': 'python'}

### Click ── コマンドラインツール作成ライブラリ

In [9]:
# Clickのインストール
%pip install Click==7.0

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
!cat greet.py

import click

@click.command()
@click.option('--words', default='Hello')
@click.argument('name')
def greet(name, words):
    click.echo(f'{words}, {name}!')

if __name__ == '__main__':
    greet()

In [11]:
!python3 greet.py rhoboro

Hello, rhoboro!


In [12]:
!python3 greet.py rhoboro --words Hi

Hi, rhoboro!


### Pillow ── 画像処理ライブラリ

In [13]:
# Pillowのインストール
%pip install Pillow==6.2.1

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import os
from PIL import Image
def thumbnail(infile, size=(128, 128)):
    outfile = os.path.splitext(
        infile)[0] + ".thumbnail"
    try:
        im = Image.open(infile)
        im.thumbnail(size)
        im.save(outfile, "JPEG")
    except IOError:
        print("cannot create thumbnail for", infile)

In [15]:
# 任意のJPEGファイルを指定する
thumbnail('dog.jpg')

# プロジェクトの作成

## Gitの利用

In [16]:
%cd workspace

/Users/suyamar/github/python-practice-book/src/13-application/workspace


In [17]:
!git init

Reinitialized existing Git repository in /Users/suyamar/github/python-practice-book/src/13-application/workspace/.git/


### .gitignoreファイルの作成

In [18]:
# gitインストール直後の場合は下記を実行してください
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

### GitHubでのソースコード管理

## パッケージのひな型作成

In [19]:
!cat requirements.txt

Click==7.0
Pillow==6.2.1
requests==2.22.0

In [20]:
%pip install -r requirements.txt

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip freeze > requirements.lock

Note: you may need to restart the kernel to use updated packages.


### lgtmパッケージの作成

In [22]:
# 空の__init__.pyを作成
# Windowsの場合は type nul > lgtm/__init__.py
!touch lgtm/__init__.py

### テストコードの作成

# 継続的インテグレーションの導入

## CircleCIでテスト自動化

### プロジェクトの追加

### config.ymlの追加

In [23]:
!cat .circleci/config.yml

version: 2
jobs:
 setup_dependencies:
   docker:
     - image: circleci/python:3.8.1
   steps:
     - checkout
     - restore_cache:
         key: deps-{{ checksum "requirements.lock" }}
     - run:
         command: |
           pip install --user -r requirements.lock
     - save_cache:
         key: deps-{{ checksum "requirements.lock" }}
         paths:
           - "~/.local"
 test:
   docker:
     - image: circleci/python:3.8.1
   steps:
     - checkout
     - restore_cache:
         key: deps-{{ checksum "requirements.lock" }}
     - run:
         command: |
           python3 -m unittest -v
workflows:
  version: 2
  all:
    jobs:
      - setup_dependencies
      - test:
          requires:
            - setup_dependencies


## テストの実行と結果の確認

# アプリケーションの開発

## コマンドライン引数の取得

### 画像ファイルのソース情報とメッセージを受け取る

In [24]:
!python3 main.py --help

Usage: main.py [OPTIONS] KEYWORD

  LGTM画像生成ツール

Options:
  -m, --message TEXT  画像に乗せる文字列  [default: LGTM]
  --help              Show this message and exit.


### テストコードの修正

## 画像の取得

### ファイルパスから画像を取得するクラスの実装

### URLから画像を取得するクラスの実装

### 検索キーワードから画像を取得するクラスの実装

### 画像を取得するクラスの利用

In [25]:
!cat lgtm/image_source.py

from io import BytesIO
import requests
from pathlib import Path

class LocalImage:
    """ファイルから画像を取得する"""

    def __init__(self, path):
        self._path = path

    def get_image(self):
        return open(self._path, 'rb')


class RemoteImage:
    """URLから画像を取得する"""

    def __init__(self, path):
        self._path = path

    def get_image(self):
        data = requests.get(self._path)
        # バイトデータをファイルオブジェクトに変換
        return BytesIO(data.content)

class RemoteImage:
    """URLから画像を取得する"""

    def __init__(self, path):
        self._url = path

    def get_image(self):
        data = requests.get(self._url)
        # バイトデータをファイルオブジェクトに変換
        return BytesIO(data.content)

class _LoremFlickr(RemoteImage):
    """キーワード検索で画像を取得する"""
    LOREM_FLICKR_URL = 'https://loremflickr.com'
    WIDTH = 800
    HEIGHT = 600

    def __init__(self, keyword):
        super().__init__(self._build_url(keyword))

    def _build_url(self, keyword):
        return (f'{self.LOREM_FLICKR_URL}/

# 画像処理

## 文字列を画像上に描画する最小限の実装例

### 文字列を中央に最適なサイズで描画する

In [26]:
!cat lgtm/drawer.py

from PIL import Image, ImageDraw, ImageFont

# 画像全体に対するメッセージ描画可能エリアの比率
MAX_RATIO = 0.8

# フォント関連の定数
FONT_MAX_SIZE = 256
FONT_MIN_SIZE = 24

# WindowsやLinuxではパスが異なる
FONT_NAME = '/Library/Fonts/Arial Bold.ttf'
FONT_COLOR_WHITE = (255, 255, 255, 0)

# アウトプット関連の定数
OUTPUT_NAME = 'output.png'
OUTPUT_FORMAT = 'PNG'


def save_with_message(fp, message):
    image = Image.open(fp)
    draw = ImageDraw.Draw(image)
    # メッセージを描画できる領域のサイズ
    # タプルの要素ごとに計算する
    image_width, image_height = image.size
    message_area_width = image_width * MAX_RATIO
    message_area_height = image_height * MAX_RATIO

    # フォントサイズを決める
    for font_size in range(FONT_MAX_SIZE, FONT_MIN_SIZE,
                           -1):
        font = ImageFont.truetype(FONT_NAME, font_size)
        # 描画に必要なサイズ
        text_width, text_height = draw.textsize(
            message, font=font)
        w = message_area_width - text_width
        h = message_area_height - text_height

        # 幅、高さともに領域内におさまる値を採用
        if w > 0 an

## 各処理の呼び出し

In [27]:
!cat lgtm/core.py

import click

from lgtm.drawer import save_with_message
from lgtm.image_source import get_image

@click.command()
@click.option('--message', '-m', default='LGTM',
              show_default=True, help='画像に乗せる文字列')
@click.argument('keyword')
def cli(keyword, message):
    """LGTM画像生成ツール"""
    lgtm(keyword, message)


def lgtm(keyword, message):
    with get_image(keyword) as fp:
        save_with_message(fp, message)

In [28]:
# 結果の画像は取得できた画像により異なる
!python3 main.py book

# コマンドとして実行する

## setup.pyの作成

In [29]:
!cat setup.py

from setuptools import find_packages, setup

setup(
    name='lgtm',
    version='1.0.0',
    packages=find_packages(exclude=('tests',)),
    install_requires=[
        'Click',
        'Pillow',
        'requests',
    ],
    entry_points={
        'console_scripts': [
            'lgtm=lgtm.core:cli'
        ]
    }
)

### entry\_points ── スクリプトインタフェースの登録を行う引数

## 動かしてみよう

In [30]:
%pip install -e .

Obtaining file:///Users/suyamar/github/python-practice-book/src/13-application/workspace
  Found existing installation: lgtm 1.0.0
    Not uninstalling lgtm at /Users/suyamar/github/python-practice-book/src/13-application/workspace, outside environment /Users/suyamar/github/python-practice-book/src/13-application/venv
    Can't uninstall 'lgtm'. No files were found to uninstall.
  Running setup.py develop for lgtm
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [31]:
# コマンドが登録された
!lgtm

Usage: lgtm [OPTIONS] KEYWORD
Try "lgtm --help" for help.

Error: Missing argument "KEYWORD".


In [32]:
!lgtm book

# 本章のまとめ